In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("cluster").getOrCreate()

     |████████████████████████████████| 212.4MB 65kB/s 
     |████████████████████████████████| 204kB 17.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=57a80de77b9b49377f7939092c99c175acb5e79f0d63c7e40b70a79eaf0f3d80
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
data=spark.read.csv("/content/hack_data.csv",inferSchema=True,header=True)

In [3]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator

In [16]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [17]:
assembler=VectorAssembler(inputCols= ['Session_Connection_Time',
                                      'Bytes Transferred',
                                      'Kali_Trace_Used',
                                      'Servers_Corrupted',
                                      'Pages_Corrupted',
                                      'WPM_Typing_Speed'],outputCol="features")

In [18]:
output=assembler.transform(data)

In [19]:
final_data=output.select("features")

In [20]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|[8.0,391.09,1.0,2...|
|[20.0,720.99,0.0,...|
|[31.0,356.32,1.0,...|
|[2.0,228.08,1.0,2...|
|[20.0,408.5,0.0,3...|
|[1.0,390.69,1.0,2...|
|[18.0,342.97,1.0,...|
|[22.0,101.61,1.0,...|
|[15.0,275.53,1.0,...|
|[12.0,424.83,1.0,...|
|[15.0,249.09,1.0,...|
|[32.0,242.48,0.0,...|
|[23.0,514.54,0.0,...|
|[9.0,284.77,0.0,3...|
|[27.0,779.25,1.0,...|
|[12.0,307.31,1.0,...|
|[21.0,355.94,1.0,...|
|[10.0,372.65,0.0,...|
|[20.0,347.23,1.0,...|
|[22.0,456.57,0.0,...|
+--------------------+
only showing top 20 rows



In [21]:
def findBestKvalue(k,dt,fc):
  kmeans=KMeans(featuresCol=fc).setK(k)
  model=kmeans.fit(dt)
  test=model.transform(dt)
  evaluator=ClusteringEvaluator()
  silhouette = evaluator.evaluate(test)
  print("k = "+ str(k) + " --> "+"Silhouette with squared euclidean distance = " + str(silhouette))


In [27]:
for i in range(2,4):
  findBestKvalue(i,final_data,"features")

k = 2 --> Silhouette with squared euclidean distance = 0.8049295570472647
k = 3 --> Silhouette with squared euclidean distance = 0.6924390056057199


In [28]:
from pyspark.ml.feature import StandardScaler

In [29]:
scalar=StandardScaler(inputCol="features",outputCol="scaledfeatures")

In [30]:
scaled_data=scalar.fit(final_data).transform(final_data)

In [31]:
#after scaled
for i in range(2,4):
  findBestKvalue(i,scaled_data,"scaledfeatures")

k = 2 --> Silhouette with squared euclidean distance = 0.6683623593283755
k = 3 --> Silhouette with squared euclidean distance = 0.30412315937808737


In [32]:
kmeans2=KMeans(featuresCol="features").setK(2)
model2=kmeans2.fit(scaled_data)
test2=model2.transform(scaled_data)
kmeans3=KMeans(featuresCol="features").setK(3)
model3=kmeans3.fit(scaled_data)
test3=model3.transform(scaled_data)

In [34]:
test3.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  144|
|         2|   91|
|         0|   99|
+----------+-----+



In [35]:
test2.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  163|
|         0|  171|
+----------+-----+

